In [2]:
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LinearRegression
pd.options.mode.chained_assignment = None  # default='warn'

## Model Fitting 

In [ ]:
def find_threshold(probabilities, true_rate):
    
    """
    Find classification threshold to calibrate model
    """
    
    index = (1-true_rate)*len(probabilities)
    
    threshold = sorted(probabilities)[int(index)]
    
    return threshold

In [4]:
def fit(clf, dta, vars_, target, scaler, classifier=True):

    X = dta[vars_]
    y = dta[target]

        
    X = scaler.fit_transform(X.values)

    try:
    
        clf.fit(X,y)
        
    except ValueError:
        
        clf = LinearRegression()
        clf.fit(X,y)
    
    if classifier:
    
        y_prob = clf.predict_proba(X)[:,1]
        
        try:
            
            threshold = find_threshold(y_prob, np.mean(y.values))
            
        except IndexError:
            
            threshold=0.5

        y_pred = [1 if p> threshold else 0 for p in y_prob]
        
    else:
            
        y_prob = []
            
        threshold = 0
            
        y_pred = clf.predict(X)
        
    
    return y_pred, y_prob, y, [clf, threshold] 

In [1]:
def fit_k_fold(clf, folds, dta, vars_, target, scaler,classifier=True):

    skf = StratifiedKFold(n_splits=folds)

    preds, probs, true, models = [],[],[],[]

    X = dta[vars_]
    y = dta[target]

    
    for train_index, test_index in skf.split(X, y):

        y_train = y.iloc[train_index]
        X_train = X.iloc[train_index]
        
        X_train = scaler.fit_transform(X_train.values)

        y_test = y.iloc[test_index]
        X_test = X.iloc[test_index]
        
        X_test = scaler.fit_transform(X_test.values)

        clf.fit(X_train, y_train)


        if classifier:
            
            y_prob = clf.predict_proba(X_test)[:,1]
            
            threshold = find_threshold(y_prob, np.mean(y.values))

            y_pred = [1 if p> threshold else 0 for p in y_prob]
            
        else:
            
            y_prob = []
            
            threshold = 0
            
            y_pred = clf.predict(X_test)

        preds.append(y_pred)
        probs.append(y_prob)
        true.append(y_test.values)
        models.append([clf, threshold])
        
    return preds, probs, true, models

## Forecasting 

In [7]:
def predict(dta, vars_, models, scenarios, scaler, classifier=True, thresholds=False):
    
    preds, probs = [],[]

    
    scaler.fit(dta[vars_].values)
    
    for var, scale_func in scenarios.items():
        
        func = scale_func[0]
        param = scale_func[1]
        
        dta[var] = func(dta[var].values,param)
        
    if len(models) == 2:
        
        models = [models]
        
    for i,model in enumerate(models):

        X = dta[vars_].values
        
        X = scaler.transform(X)

        clf = model[0]
        
        threshold = model[1]

        if classifier:
            
            y_prob = clf.predict_proba(X)[:,1]
            
            if thresholds:
    
                y_pred = [1 if p> threshold else 0 for p in y_prob]
        
            else:
                
                y_pred = clf.predict(X)
        
        else:
            
            y_prob = []
            
            y_pred = clf.predict(X)
    
        preds.append(y_pred)
        probs.append(y_prob)
        
    return preds, probs

## Functions for scenarios 

In [2]:
def imd(vals, n):
    
    new_vals = np.array(vals)-n
    
    return new_vals
        
def income(vals, n):
    
    new_vals = np.array(vals)-n
    
    return new_vals

def housing(vals, n):
    
    new_vals = np.array(vals)-n
    
    return new_vals

def living(vals, n):
    
    new_vals = np.array(vals)-n
    
    return new_vals

## FitPredict 

In [6]:
def fit_predict_classifier(dta_train, vars_, target, dta_forecast):
    
    clf = LogisticRegression(max_iter=10000)
    folds = 5
    scaler = StandardScaler()
    
    
    _, _, _, models = fit_k_fold(clf, folds, dta_train, vars_, target + '_binary', scaler, classifier=True)
    
    #probabilities for training regression
    scenarios = {}
    
    preds, probs = predict(dta_train, vars_, models, scenarios, scaler, classifier=True)
    
    dta_train[target + '_binary_pred'] = np.mean(preds, axis=0).round(0)
    dta_train[target + '_binary_prob'] = np.mean(probs, axis=0)
    
    
    #business as usual
    scenarios = {}
    
    preds, probs = predict(dta_forecast, vars_, models, scenarios, scaler, classifier=True)
    
    dta_forecast[target + '_no_change_binary'] = np.mean(preds, axis=0).round(0)
    dta_forecast[target + '_no_change_binary_prob'] = np.mean(probs, axis=0)
    
    
    #imd decrease by 1
    scenarios = {'wd_imd_decile_19':[imd,1]}
    
    preds, probs = predict(dta_forecast, vars_, models, scenarios, scaler, classifier=True)
    
    dta_forecast[target + '_imd_m1_binary'] = np.mean(preds, axis=0).round(0)
    dta_forecast[target + '_imd_m1_binary_prob'] = np.mean(probs, axis=0)
    
    return dta_train, dta_forecast

In [ ]:
def fit_predict_regression(dta_train, vars_, target, dta_forecast):
    
    scaler = StandardScaler()
    clf = TweedieRegressor(power=1, max_iter=10000)
    
    _, _, _, models = fit(clf, dta_train, vars_ + [target + '_binary_prob'], target, scaler, classifier=False)
    
    #probabilities for training regression
    scenarios = {}
    
    preds, probs = predict(dta_train, vars_ +  [target + '_binary_prob'],
                           models, scenarios, scaler, classifier=False)
    
    preds = [max(p,0) for p in preds[0]]
    
    dta_train[target + '_pred'] = preds

    
    #business as usual
    scenarios = {}

    preds, probs = predict(dta_forecast, vars_ + [target + '_no_change_binary_prob'], 
                           models, scenarios, scaler, classifier=False)
    
    preds = [max(p,0) for p in preds[0]]
    
    dta_forecast[target + '_no_change'] = preds

        
    #imd decrease by 1
    
    scenarios = {'wd_imd_decile_19':[imd,1]}
    
    preds, probs = predict(dta_forecast, vars_ + [target + '_imd_m1_binary_prob'],
                           models, scenarios, scaler, classifier=False)
    
    preds = [max(p,0) for p in preds[0]]

    dta_forecast[target + '_imd_m1'] = preds
    
    return dta_train, dta_forecast

## Model per imd 

In [ ]:
def fit_predict_regression_imd(dta_train, vars_, target, dta_forecast):
    
    scaler = StandardScaler()
    clf = TweedieRegressor(power=1., max_iter=100000)
    
    _, _, _, models = fit(clf, dta_train, vars_ + [target + '_binary_prob'], target, scaler, classifier=False)
    
    #probabilities for training regression
    scenarios = {}
    
    preds, probs = predict(dta_train, vars_ +  [target + '_binary_prob'],
                           models, scenarios, scaler, classifier=False)
    
    preds = [max(p,0) for p in preds[0]]
    
    dta_train[target + '_pred'] = preds

    
    #business as usual
    scenarios = {}

    preds, probs = predict(dta_forecast, vars_ + [target + '_no_change_binary_prob'], 
                           models, scenarios, scaler, classifier=False)
    
    preds = [max(p,0) for p in preds[0]]
    
    dta_forecast[target + '_no_change'] = preds

    
    return dta_train, dta_forecast, models

In [ ]:
def fit_predict_classifier_imd(dta_train, vars_, target, dta_forecast):
    
    clf = LogisticRegression(max_iter=10000)
    scaler = StandardScaler()
    
    _, _, _, models = fit(clf, dta_train, vars_, target + '_binary', scaler, classifier=True)
    
    #probabilities for training regression
    scenarios = {}
    
    preds, probs = predict(dta_train, vars_, models, scenarios, scaler, classifier=True)
    
    dta_train[target + '_binary_pred'] = np.mean(preds, axis=0).round(0)
    dta_train[target + '_binary_prob'] = np.mean(probs, axis=0)
    
    
    #business as usual
    scenarios = {}
    
    preds, probs = predict(dta_forecast, vars_, models, scenarios, scaler, classifier=True)
    
    dta_forecast[target + '_no_change_binary'] = np.mean(preds, axis=0).round(0)
    dta_forecast[target + '_no_change_binary_prob'] = np.mean(probs, axis=0)
    
    
    return dta_train, dta_forecast, models

In [ ]:
def predict_scenario_imd(vars_, dta_forecast, target, classifiers, regressions, scenarios):
    
    scaler = StandardScaler()
    scenarios = {}
    forecast = pd.DataFrame()
    
    for imd in np.arange(1,11):
        
        forecast_imd = dta_forecast.loc[dta_forecast.wd_imd_decile_19==imd]
        
        if imd==1:
            
            forecast_imd[target + '_imd_m1'] = forecast_imd[target + '_no_change'].values
            forecast_imd[target + '_imd_m1_binary'] = forecast_imd[target + '_no_change_binary'].values
            forecast_imd[target + '_imd_m1_binary_prob'] = forecast_imd[target + '_no_change_binary_prob'].values
            forecast=pd.concat([forecast,forecast_imd])
            
        else:
            
            forecast_imd['wd_imd_decile_19'] = forecast_imd['wd_imd_decile_19'] - 1
            
            preds, probs = predict(forecast_imd, vars_, classifiers[imd-1], scenarios, 
                                   scaler, classifier=True, thresholds=True)
            
            forecast_imd[target + '_imd_m1_binary'] = np.mean(preds, axis=0).round(0)
            forecast_imd[target + '_imd_m1_binary_prob'] = np.mean(probs, axis=0)
            
            
            preds, probs = predict(forecast_imd, vars_ + [target + '_imd_m1_binary_prob'], 
                           regressions[imd-1], scenarios, scaler, classifier=False)
            
            preds = [max(p,0) for p in preds[0]]
            
            forecast_imd[target + '_imd_m1'] = preds
            
            forecast_imd['wd_imd_decile_19'] = forecast_imd['wd_imd_decile_19'] + 1
    
            forecast=pd.concat([forecast, forecast_imd])
            
    return forecast

In [ ]:
def fit_imd(dta_train, vars_, target, dta_forecast):
    
    train, forecast = pd.DataFrame(), pd.DataFrame()
    classifiers, regressions = {},{}
    
    for imd in np.arange(1,11):
        
        train_imd = dta_train.loc[dta_train.wd_imd_decile_19==imd]
        forecast_imd = dta_forecast.loc[dta_forecast.wd_imd_decile_19==imd]
        
        train_imd, forecast_imd, models = fit_predict_classifier_imd(train_imd, vars_, target, forecast_imd)
        
        classifiers[imd] = models
        
        train_imd, forecast_imd, models = fit_predict_regression_imd(train_imd, vars_, target, forecast_imd)
        
        regressions[imd] = models
        
        train = pd.concat([train, train_imd])
        forecast = pd.concat([forecast, forecast_imd])
        
    return train, forecast, classifiers, regressions

In [ ]:
def fit_predict_imd(dta_train, vars_, target, dta_forecast):
    
    dta_train, dta_forecast, classifiers, regressions =\
    fit_imd(dta_train, vars_, target, dta_forecast)
    
    scenarios = {}
    
    dta_forecast = predict_scenario_imd(vars_, dta_forecast, target, classifiers, regressions, scenarios)
    
    #save variable importances
    coefs = pd.DataFrame()    
    coefs['vars'] = vars_ + [target + '_binary_prob']
    
    for imd in np.arange(1,11):

        coefs[f'lr_{imd}'] = list(classifiers[imd][0].coef_[0]) + ['--']
        coefs[f'poisson_{imd}'] = regressions[imd][0].coef_
        
    coefs.to_csv(f'{target}_coefficients.csv')
    
    return dta_train, dta_forecast

## All models 

In [ ]:
def cross_validate(dta_train, vars_, targets, folds):
    
    results = pd.DataFrame(index = targets)

    kf = KFold(n_splits=folds)
    kf.get_n_splits(dta_train)

    count=1

    for train_index, test_index in kf.split(dta_train):

        train = dta_train.iloc[train_index]
        test = dta_train.iloc[test_index]

        mae_= []
        
        for t in targets:

            print(f'Starting {t}, fold {count}')

            train, test = fit_predict_imd(train, vars_, t, test)

            mae_.append(mae(test[t], test[t+'_no_change']))

        results[f'{count}_mae'] = mae_

        print(f'{count} folds completed \n\n')

        count+=1
        
    results['2021 Total'] = dta_train[targets].sum().astype(int)
        
    mae_ = [f'{k}_mae' for k in range(1,6)]

    results['K-fold MAE'] = [f'{round(D,2)} +/- {round(results[mae_].std(axis=1).values[i],2)}' 
                              for i,D in enumerate(results[mae_].mean(axis=1).values)]
    
    results = results.drop(mae_, axis=1)
    
    return results

In [ ]:
def predict_all(dta_train, dta_forecast, targets, vars_, results):
    
    count=0

    for t in targets:
    
        print(f'Starting {t}')
    
        dta_train, dta_forecast = fit_predict_imd(dta_train, vars_, t, dta_forecast)
    
    
        count+=1
    
        print(f'{count} pods completed \n\n')
        
    no_change = [t + '_no_change' for t in targets]
    imd_m1 = [t + '_imd_m1' for t in targets]
    last_year = [t + '_pred' for t in targets]

    ## Results

    scores = []
    for t in targets:
        scores.append(mae(dta_train[t], dta_train[t+'_pred']))

    results['Training MAE'] = scores

    results['2021 Predicted'] = dta_train[last_year].sum().astype(int).values
    results['2022 Baseline'] = dta_forecast[no_change].sum().astype(int).values
    results['CoL'] = dta_forecast[imd_m1].sum().astype(int).values

    results['% change CoL'] = (results['CoL'] - results['2022 Baseline'])*100/results['2022 Baseline']
    
    return results